In [10]:
import numpy as np

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models.widgets import Slider
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

    
t_in = np.array([0., 1.])
f_in = np.array([2., 3.])
diff_in = np.array([0., 0.])

# cubic polynomialS
polyDeg = 3

A = np.zeros([polyDeg + 1, polyDeg + 1])
for i in np.arange(0, polyDeg + 1):
    A[[0, 1], i] = np.power(t_in, polyDeg - i)

A[2, :] = np.array([3 * np.square(t_in[0]), 2 * t_in[0], 1, 0])
A[3, :] = np.array([3 * np.square(t_in[1]), 2 * t_in[1], 1, 0])

t = np.arange(0, 1, 0.01)

# Calculate polynomial
cubicParams = np.dot(np.linalg.inv(A), np.append(f_in, diff_in))
y = np.polyval(cubicParams, t)
    
source = ColumnDataSource(data=dict(t=t, y=y))

# Set up Plot
plot = figure(plot_width=400, plot_height=400, x_range=[-0.3,1.3], y_range=[0.7,4.3])
plot.line('t', 'y', source=source, line_width=3, line_alpha=0.6)
#plot.circle(t_in, f_in, size=8, fill_color="white")

# Controls
f1_slider = Slider(start=1, end=4, value=2, step=0.01, title="f(0)")
f2_slider = Slider(start=1, end=4, value=3, step=0.01, title="f(1)")
diff1_slider = Slider(start=-10, end=10, value=0, step=0.01, title="f'(1)")
diff2_slider = Slider(start=-10, end=10, value=0, step=0.01, title="f'(2)")

def update_data(attrname, old, new):
    
    # Get inputs
    f_in[0] = f1_slider.value
    f_in[1] = f2_slider.value

    diff_in[0] = diff1_slider.value
    diff_in[1] = diff2_slider.value
    
    # Generate new curve
    cubicParams = np.dot(np.linalg.inv(A), np.append(f_in, diff_in))
    y = np.polyval(cubicParams, t)
    
    source.data = dict(t=t, y=y)
    
    source.change.emit()
    push_notebook()
    
    
for widget in [f1_slider, f2_slider, diff1_slider, diff2_slider]:
    widget.on_change('value', update_data)
    
    
# Set up layouts
inputs = widgetbox(f1_slider, f2_slider, diff1_slider, diff2_slider)
layout = row(plot, inputs)

output_notebook()
show(layout)

Loading BokehJS ...